In [1]:
import cv2
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', 
 path='platt.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\muham/.cache\torch\hub\master.zip
YOLOv5  2023-3-19 Python-3.9.16 torch-1.13.1 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [8]:
import cv2
import numpy as np
import time
import xlsxwriter
import pandas as pd

workbook = xlsxwriter.Workbook('Count.xlsx')
worksheet = workbook.add_worksheet()
workbook.close()
df = pd.read_excel('Count.xlsx')

date = []
number = []
pallet = []

w_min=10 #Largura minima do retangulo
h_min=10 #Altura minima do retangulo

offset=3 #Erro permitido entre pixel  

#y1=550 #Posição da linha de contagem 

delay=60
detec=[]
carr=0

def pega_centro(x, y, w, h):
    
    x1 = int(int(w-x) / 2)
    y1 = int(int(h-y) / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

cap = cv2.VideoCapture('Video.mp4')

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out= cv2.VideoWriter('out.mp4', cv2.VideoWriter_fourcc(*'XVID'), 60, (width,height))

while True:
    ret , frame1 = cap.read()
    if frame1 is None:
            break
    #cuda.torch= True 
    if cv2.waitKey(1)==ord('q'):
            break
    #frame1=cv2.resize(frame1,(1020,600)) 
    cv2.line(frame1, (337,550),(1720 , 550), (255,127,0), 3) 
    #cv2.line(frame1, (177, 330), (920, 330), (255,127,0), 3) 
    #grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(frame1,(3,3),10)
    results = model(blur, size=640)  
    res=results.pandas().xyxy[0]
    for i in range (len(res)) :
        xmin,ymin,xmax,ymax,confidence,_,_=res.iloc[i,:]
        xmin,ymin,xmax,ymax=int(xmin),int(ymin),int(xmax),int(ymax)
        if confidence > 0.1 :
            cv2.rectangle(frame1, (xmin , ymin), (xmax , ymax), (0,127,255), 2)
            cv2.putText(frame1, str("paltt of water"), (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX,1, ((0, 255, 0)), 2, cv2.LINE_AA) 
            cv2.putText(frame1, str(round(confidence,2)), (xmin+225 , ymin-10), cv2.FONT_HERSHEY_SIMPLEX,1, ((0, 255, 0)), 2, cv2.LINE_AA)
            #cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)
            
            centro = pega_centro(xmin , ymin,xmax , ymax)
            
            detec.append(centro)
            #print(centro)
            cv2.circle(frame1, centro, 6, (0, 0,255), -1)
            for(x,y) in detec:
                if(y<(553))and(y>(547)):
                        carr+=1
                        cv2.line(frame1, (337,550),(1720 , 550), (0,127,255), 3)
                        detec.remove((x,y))
                        print("no. of pallets detected : " + str(carr))
                        _, frame = cap.read()
                        time_string = time.strftime("%m/%d/%Y, %I:%M:%S", time.localtime())
                        date.append(time_string)
                        number.append(carr)
                        pallet.append(1)
                      
            cv2.putText(frame1, "PALLETS COUNT : "+str(carr), (450,48), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
        
    
    cv2.imshow("Video Original" , frame1)
    out.write(frame1)
    k = cv2.waitKey(10)
    if k == 27 :
        break
   
    
df['time'] = date
df['Pallet'] = pallet
df['Total number of pallets'] = number  
df.to_excel('Count.xlsx')        
    
        

cv2.destroyAllWindows()
cap.release()
out.release()

In [7]:
#check pixels in the frame
def click(ev,x,y,flags,param):
        if ev==cv2.EVENT_LBUTTONDOWN:
                print(x,' , ',y)
                font=cv2.FONT_HERSHEY_SIMPLEX
                strxy=str(x)+' , '+str(y)
                cv2.putText(frame,strxy,(x,y),font, .5,(255,255,0),2)
                cv2.imshow('image',frame)
import cv2                
cap=cv2.VideoCapture('hiv00005.mp4')

while True:
    
    ret,frame=cap.read()
    if not ret:
        break
    #frame=cv2.resize(frame,(1020,600))

   
    #img=cv2.imread('unnamed.jpg')
    
    cv2.imshow('image',frame)
    cv2.setMouseCallback('image',click)
        
    cv2.line(frame, (177, 348), (920, 348), (255,127,0), 3)
                
    #cv2.imshow("FRAME",frame)
    cv2.imshow('image',frame)
    #cv2.setMouseCallback("FRAME",POINTS)
   

    if cv2.waitKey(1)==ord('q'):
            break
cap.release()


cv2.destroyAllWindows()


554  ,  0
850  ,  301
